<a href="https://colab.research.google.com/github/cluckey-smith/VinBigData_comp_repo/blob/main/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import pandas as pd
import numpy as np
import cv2
from google.colab import drive
drive.mount('/content/drive')
import os
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
def rgb2gray(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

In [2]:
df = pd.read_csv('./train.csv')

In [3]:
train_files = os.listdir('/content/drive/MyDrive/Vin_big_data/train/')

In [59]:
labels = []
x_min_list = []
x_max_list = []
y_min_list = []
y_max_list = []
img_name = []
img = []
bounding_boxes = []
for i,r in df[:20].iterrows():
  if (str(r['image_id']) + '.png') in train_files:
    img_name.append(('/content/drive/MyDrive/Vin_big_data/train/') + r['image_id'] + '.png')
    image = load_img(('/content/drive/MyDrive/Vin_big_data/train/') + r['image_id'] + '.png')
    image = img_to_array(image)
    if len(image.shape) > 2:
      image = rgb2gray(image) # Not every image is in grayscale. This is converting them to it
    h,w = image.shape[:2]
    image = image / 255 # Converting our graycale image to values between 0 and 1
    img.append(image)
    labels.append(r['class_name'])
    x_min = r['x_min'] / w
    x_max = r['x_max'] / w
    y_min = r['y_min'] / h
    y_max = r['y_max'] / h
    x_min_list.append(x_min)
    x_max_list.append(x_max)
    y_min_list.append(y_min)
    y_max_list.append(y_max)
    bounding_boxes.append((x_min, y_min, x_max, y_max))
    

In [60]:
bounding_boxes

[(nan, nan, nan, nan),
 (nan, nan, nan, nan),
 (1.349609375, 2.685546875, 3.228515625, 3.576171875),
 (2.46875, 1.451171875, 3.146484375, 1.990234375),
 (nan, nan, nan, nan),
 (1.224609375, 0.697265625, 1.849609375, 0.845703125),
 (2.630859375, 0.478515625, 4.2734375, 4.236328125),
 (1.087890625, 4.59375, 1.318359375, 4.8515625),
 (2.595703125, 1.451171875, 2.970703125, 1.87109375),
 (3.626953125, 3.138671875, 4.15234375, 3.9765625),
 (1.171875, 2.6015625, 1.763671875, 2.974609375),
 (0.767578125, 0.552734375, 1.60546875, 1.255859375),
 (nan, nan, nan, nan),
 (nan, nan, nan, nan),
 (2.5234375, 1.08203125, 2.884765625, 1.572265625),
 (nan, nan, nan, nan),
 (nan, nan, nan, nan),
 (0.646484375, 0.90234375, 2.703125, 4.619140625),
 (3.193359375, 1.314453125, 3.86328125, 2.248046875),
 (2.71484375, 1.71484375, 3.15625, 2.271484375)]

In [25]:
df.groupby(['image_id', 'class_name']).count()

class_id  ...  img_name
image_id                         class_name                    ...          
000434271f63a053c4128a0ba6352c7f No finding                 3  ...         3
00053190460d56c53cc3e57321387478 No finding                 3  ...         3
0005e8e3701dfb1dd93d53e2ff537b6e Consolidation              1  ...         1
                                 Infiltration               1  ...         1
                                 Lung Opacity               2  ...         2
...                                                       ...  ...       ...
ffe6f9fe648a7ec29a50feb92d6c15a4 Other lesion               1  ...         1
ffea246f04196af602c7dc123e5e48fc No finding                 3  ...         3
ffeffc54594debf3716d6fcd2402a99f Aortic enlargement         3  ...         3
fff0f82159f9083f3dd1f8967fc54f6a No finding                 3  ...         3
fff2025e3c1d6970a8a6ee0404ac6940 No finding                 3  ...         3

[25971 rows x 7 columns]

In [27]:
df[df['image_id'] == '0005e8e3701dfb1dd93d53e2ff537b6e']

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,img_name
664,0005e8e3701dfb1dd93d53e2ff537b6e,Lung Opacity,7,R10,900.0,587.0,1205.0,888.0,
13648,0005e8e3701dfb1dd93d53e2ff537b6e,Nodule/Mass,8,R8,932.0,567.0,1197.0,896.0,
15180,0005e8e3701dfb1dd93d53e2ff537b6e,Infiltration,6,R10,900.0,587.0,1205.0,888.0,
16938,0005e8e3701dfb1dd93d53e2ff537b6e,Lung Opacity,7,R9,905.0,583.0,1203.0,890.0,
48751,0005e8e3701dfb1dd93d53e2ff537b6e,Consolidation,4,R8,932.0,567.0,1197.0,896.0,
